In [23]:
%reset -f

import re
import aiohttp
import asyncio
import requests
from configparser import ConfigParser
from bs4 import BeautifulSoup

header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
}

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

''' Latest scraping version functions'''
async def fetch(url):
    async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(ssl=False), headers=header) as session:
        async with session.get(url) as response:
            if response.status == 200:
                return await response.text()
        
async def crawl(parsers, url_type):
    output = []

    used_tools = {k:v for k,v in parsers.items(url_type)}

    urls = list(used_tools.values())
    tools = list(used_tools.keys())
    
    tasks = [fetch(url) for url in urls]
    
    html_pages = await asyncio.gather(*tasks)
    
    for i, html in enumerate(html_pages):
        tag_n = eval(parsers['PARSERS'][tools[i] + "_tag_n"])
        tag_p = eval(parsers['PARSERS'][tools[i] + "_tag_p"])
        tag_s = eval(parsers['PARSERS'][tools[i] + "_tag_s"])
        attr_n = eval(parsers['PARSERS'][tools[i] + "_attr_n"])
        attr_p = eval(parsers['PARSERS'][tools[i] + "_attr_p"])
        attr_s = eval(parsers['PARSERS'][tools[i] + "_attr_s"])
        re_n = eval(parsers['PARSERS'][tools[i] + "_re_n"])
        re_p = eval(parsers['PARSERS'][tools[i] + "_re_p"])
        re_s = eval(parsers['PARSERS'][tools[i] + "_re_s"])
        
        soup = BeautifulSoup(html, 'lxml')
        name = soup.find_all(tag_n, **attr_n)
        price = soup.find_all(tag_p, **attr_p)
        stock = soup.find_all(tag_s, **attr_s)
        
        ''' In case we need some regexp to extract data'''
        if re_n != 'False':
            name = re.findall(re_n, str(name))
        if re_p != 'False':
            price = re.findall(re_p, str(price))
        if re_s != 'False':
            stock = re.findall(re_s, str(stock))

        if type(price[0]) == tuple:
            price = [item for sublist in price for item in sublist if item]
        #print(name)
        for j in range(len(name)):
            formatted_web = bcolors.BOLD + tools[i].upper() + bcolors.ENDC
            if re_n != 'False':
                formatted_name = bcolors.BOLD + name[j].strip() + bcolors.ENDC
            else:
                formatted_name = bcolors.BOLD + name[j].get_text().strip() + bcolors.ENDC
            if re_p != 'False':
                formatted_price = bcolors.OKBLUE + price[j].strip().replace('&nbsp;', ',') + bcolors.ENDC
            else:
                formatted_price = bcolors.OKBLUE + price[j].get_text().strip().replace('&nbsp;', ',') + bcolors.ENDC        
            if re_s != 'False':
                formatted_stock = bcolors.WARNING + stock[j].strip() + bcolors.ENDC
            else:
                formatted_stock = bcolors.WARNING + stock[j].get_text().strip() + bcolors.ENDC
            output.append([formatted_web, formatted_name, formatted_price, formatted_stock])
            # print(formatted_name)
    for row in output:
        print("{: <20} {: <100} {: >20} {: <40}".format(*row))
    
async def main():  
    parsers = ConfigParser(delimiters=(':'))
    parsers.read('../parsers.ini')
        
    '''Getting latest versions from webs'''
    await crawl(parsers, 'URLS')
        
await main()

MSI          MSI GeForce RTX 5070 Ti 16G VANGUARD SOC LAUNCH EDITION | Tarjeta Gráfica                      1.149,90€ Agotado                        
MSI          MSI GeForce RTX 5070 Ti 16G GAMING TRIO OC PLUS | Tarjeta Gráfica                              1.114,90€ Agotado                        
MSI          MSI GeForce RTX 5070 Ti 16G VENTUS 3X OC | Tarjeta Gráfica                                       889,90€ Agotado                        
CASEKING     Tarjeta Gráfica Asus GeForce® RTX 5070 Ti TUF Gaming OC 16GB GDDR7 DLSS4                      1.170,51 € Agotado                        
CASEKING     Tarjeta Gráfica Gigabyte GeForce® RTX 5070 Ti Aero OC 16GB GDDR7 DLSS4                        1.170,51 € Agotado                        
CASEKING     Tarjeta Gráfica Gigabyte GeForce® RTX 5070 Ti Eagle OC SFF 16GB GDDR7 DLSS4                   1.062,30 € Agotado                        
CASEKING     Tarjeta Gráfica Gigabyte GeForce® RTX 5070 Ti Eagle OC Ice SFF 16GB GDDR7 DLSS4        